In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from rich.traceback import install as install_traceback

install_traceback()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f9e102496c0>>

In [82]:
# Set for latex font and setting
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# Load in the data

In [2]:

import glob

In [3]:
exp_data_path = '../experiment_data/direct-opt-global-INIT_FAIL_PERTUBATIONS_return-to-origin'
exp_data_file_name = 'coeff_evo_fail_creepy-incantation-7_purtstr1_run*.csv'
file_paths = glob.glob(f'{exp_data_path}/{exp_data_file_name}')



In [6]:
# Create an empty dictionary to store dataframes for each lineKey
linekey_dataframes = {}

# Load and process each CSV file
for run_i, file_path in enumerate(file_paths):
    # Load CSV file into a pandas dataframe
    df = pd.read_csv(file_path)
    step_col = df['step']

    # Iterate over each unique value in the 'linekey' column
    for linekey in df['lineKey'].unique():
        # Create a dataframe for the current lineKey if not already present
        if linekey not in linekey_dataframes:
            linekey_dataframes[linekey] = pd.DataFrame()
            linekey_dataframes[linekey]['step'] = df[df['lineKey'] == linekey]['step']

        # Filter the original dataframe for the current linekey
        line_df = df[df['lineKey'] == linekey]

        # Add 'lineval' column to the dataframe
        linekey_dataframes[linekey][f'lineVal_run{run_i}'] = line_df['lineVal']
    
    # Iterate over each lineKey dataframe and calculate mean, min, max
    for linekey, df in linekey_dataframes.items():

        # Calculate mean, min, and max for 'lineval' columns
        linekey_dataframes[linekey]['lineVal_mean'] = df.filter(like='lineVal').mean(axis=1)
        linekey_dataframes[linekey]['lineVal_min'] = df.filter(like='lineVal').min(axis=1)
        linekey_dataframes[linekey]['lineVal_max'] = df.filter(like='lineVal').max(axis=1)


# Plot the lines

In [77]:
import seaborn as sns
from matplotlib.patches import Rectangle
likekey_palette = sns.color_palette('husl', n_colors=len(linekey_dataframes))

In [83]:
df = linekey_dataframes[linekey]

# Plot line
ax = plt.axes()
for i, (linekey, df) in enumerate(linekey_dataframes.items()):
    line_color = likekey_palette[i]
    area_color = sns.desaturate(line_color, 0.5)
    ax.plot(df['step'], df['lineVal_mean'],label='$\mathbf{L}_%d$'%i, color=line_color, linewidth=1)
    ax.fill_between(df['step'], df['lineVal_min'], df['lineVal_max'], color=area_color, alpha=0.2)

    
# zoom around origin
# Add an inset plot (picture-in-picture) for zoomed version around (0.2, 0)
axins = plt.axes([0.62, 0.16, 0.25, 0.25])  # [x, y, width, height]
for i, (linekey, df) in enumerate(linekey_dataframes.items()):
    line_color = likekey_palette[i]
    area_color = sns.desaturate(line_color, 0.5)
    axins.plot(df['step'], df['lineVal_mean'], color=line_color)
    axins.fill_between(df['step'], df['lineVal_min'], df['lineVal_max'], color=area_color, alpha=0.2)

axins.set_xlim(0.15, 0.25)
axins.set_ylim(-0.1, 0.1)
axins.tick_params(axis='both', which='both', labelsize=8)
    
    
# zero line
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axins.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Set up axis
ax.set_xlabel('Epoch')
ax.set_ylabel('$\mathbf{L}_i$')
ax.set_xlim(0,1)
ax.set_ylim(-1,1)

# Draw a rectangle around the zoomed-in area in the original axes
rect = Rectangle((0.15, -0.1), 0.1, 0.2, linewidth=1, edgecolor='red', facecolor='none')
ax.add_patch(rect)

ax.legend(loc='upper right', ncol=3, markerscale=1.5, fancybox=True, edgecolor='black')
plt.show()

/tmp/ipykernel_23754/601919782.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [84]:
# Save figure
output_dir = "/home/dtetruash/Thesis/report/figures/direct_opt"


In [85]:
plt.savefig(f'{output_dir}/sh_init_non-neg_return-to-origin.pgf')